In [1]:
import pandas as pd
import numpy as np
import os
from datetime import datetime,date,timedelta,timezone
import calendar
import json


from google.cloud import bigquery
from google.oauth2 import service_account
from google.cloud.exceptions import NotFound
from google.api_core.exceptions import BadRequest



In [2]:
collectionDate='2023-09-02 00:00' # comment
# uncomment and indent
# import functions_framework
# @functions_framework.http
# def collect_prediction_result(request):   # run on clound function

# def collect_prediction_result(collectionDate): # migrate


# Dict To store all collective data Mode To run Job

In [3]:
dictCollectPerf={}

# uncomment
mode=1 # 2 for prodictoin 1 for test/migrate 
#modelList=['spy-ema1-60t10-ds0115t0523','qqq-ema1-30t5-ds0115t0523','spy-signal-60t10-ds0115t0523']

# comment
#model_id='spy-ema1-60t10-ds0115t0523'
model_id='qqq-ema1-30t5-ds0115t0523'
#model_id="spy-signal-60t10-ds0115t0523"

# Init parameter

In [4]:
if mode==1: # Migrate to backfill data and Test 
    logDate=collectionDate
    log_date=datetime.strptime(logDate,'%Y-%m-%d %H:%M')
    log_timestamp=datetime.strptime(logDate,'%Y-%m-%d %H:%M')
else: # On weekly basis
    log_timestamp=datetime.now(timezone.utc)
    log_date=datetime.strptime(log_timestamp.strftime('%Y-%m-%d'),'%Y-%m-%d')

week_day=log_date.weekday()
day_name=calendar.day_name[log_date.weekday()]

print(f"Date to collect data on {log_date.strftime('%Y-%m-%d')} {day_name}(Idx:{week_day}) at {log_timestamp}")
#week_day=log_date.weekday() Sature=5  [0,1,2,3,4,5,6]

if  week_day==5:
    last_trading_day_of_week=1
else:
    # comment
    raise Exception("Saturday is allowed  as Collection Date for forcasting result.")   
    # uncomment
    # return "Saturday is allowed  as Collection Date for forcasting result."  

print(f"week_day={week_day} and last_trading_day_of_week={last_trading_day_of_week}")

genTableSchema=True 
metric_name='mae'


Date to collect data on 2023-09-02 Saturday(Idx:5) at 2023-09-02 00:00:00
week_day=5 and last_trading_day_of_week=1


# Create Start to End Date By Getting Last Date of Week

In [5]:
# get  prev prediction  from  get end prediction to beginneg or predicton of week 
endX=log_date+timedelta(days=-last_trading_day_of_week)
startX=endX+timedelta(days=1)+timedelta(days=-5)
print(f"Collection data from {startX.strftime('%A %d-%m-%Y')} to {endX.strftime('%A %d-%m-%Y')}")

endX=endX.strftime('%Y-%m-%d')
startX=startX.strftime('%Y-%m-%d')

print(f"Convert Start and End Date to gather data {startX} - {endX} to string")

Collection data from Monday 28-08-2023 to Friday 01-09-2023
Convert Start and End Date to gather data 2023-08-28 - 2023-09-01 to string


# BigQuery Setting & Configuration Variable

In [129]:
date_col='date'
projectId='pongthorn'
dataset_id='FinAssetForecast'

table_data_id=f"{projectId}.{dataset_id}.fin_data"
table_id = f"{projectId}.{dataset_id}.fin_movement_forecast"
table_model_id= f"{projectId}.{dataset_id}.model_ts_metadata"

table_perf_id= f"{projectId}.{dataset_id}.model2_demo_forecast_performance"

print(table_id)
print(table_data_id)
print(table_model_id)
print(table_perf_id)

client = bigquery.Client(project=projectId )

def load_data_bq(sql:str):
    query_result=client.query(sql)
    df=query_result.to_dataframe()
    return df

pongthorn.FinAssetForecast.fin_movement_forecast
pongthorn.FinAssetForecast.fin_data
pongthorn.FinAssetForecast.model_ts_metadata
pongthorn.FinAssetForecast.model2_demo_forecast_performance


# Start Loop

In [130]:
# uncomment  and indent
# def process_data(model_id):
print(f"Collect data : {model_id} ")

Collect data : qqq-ema1-30t5-ds0115t0523 


# Check where the given date collected data or not?

In [131]:
# this verions , it will check once , it is assumes that  all models have been collected data at once.
# sqlCheck=f"""
# select collection_timestamp from `{table_perf_id}`
# where date(collection_timestamp)='{log_date.strftime('%Y-%m-%d')}'
# """
# this version , it will check on each model invidually
# you can move it inside def process

# script in the first run because we want to generate table with nested and repeated column
# sqlCheck=f"""
# select collection_timestamp from `{table_perf_id}`
# where date(collection_timestamp)='{log_date.strftime('%Y-%m-%d')}' and model_id='{model_id}'
# """

# print(sqlCheck)
# dfCheckDate=load_data_bq(sqlCheck)
# if  dfCheckDate.empty==False:
#     print(f"Collection data on {log_date} for {model_id} found, no any action")
#     # uncomment
#     #return f"Collection data on {log_date} for {model_id} found, no any action"
# else:
#     print(f"We are ready to Collect data on {log_date}")

# Get Model Meta

In [132]:
def get_model_metadata(model_id):
    sqlModelMt=f"""
    SELECT * FROM `{table_model_id}`  where model_id='{model_id}'
    """
    print(sqlModelMt)
    dfModelMeta=load_data_bq(sqlModelMt)
    return  dfModelMeta

dfModelMeta=get_model_metadata(model_id)

if dfModelMeta.empty==False:
    modelMeta=dfModelMeta.iloc[0,:]
    print(modelMeta)
    asset_name=modelMeta['asset']
    prediction=modelMeta['prediction']
else: 
    raise Exception(f"Not found model id  {model_id}")


    SELECT * FROM `pongthorn.FinAssetForecast.model_ts_metadata`  where model_id='qqq-ema1-30t5-ds0115t0523'
    
model_id                                         qqq-ema1-30t5-ds0115t0523
asset                                                                  QQQ
prediction                                                            EMA1
input_sequence_length                                                   30
output_sequence_length                                                   5
gs_model_path               gs://demo-ts-forecast-pongthorn/model_qqq_ema1
local_model_path                                      model/model_qqq_ema1
model_file                         EMA1_30To5_QQQ_E200S20-Y2015-2023_ma.h5
scaler_file                    scaler_EMA1_30To5_QQQ_E200S20-Y2015-2023.gz
scaler_pred_file          scaler_pred_EMA1_30To5_QQQ_E200S20-Y2015-2023.gz
Name: 0, dtype: object


# Retrive forecasting result data to Dictionary

In [133]:
def get_forecasting_result_data(request):

    if   request is not None:  
        start_date=request["start_date"]
        end_date=request["end_date"]
        prediction_name=request["prediction_name"]
        asset_name=request["asset_name"]
        model_id=request["model_id"]
    else:
        raise Exception("No request parameters such as start_date,prediction_name,asset_name")

    
    print("1.How far in advance does model want to  make prediction")
    sqlOutput=f"""
    select t.prediction_date, t.pred_timestamp,t.asset_name,t.prediction_name,
    t_pred.output_date as {date_col},t_pred.output_value as {prediction_name}
    from  `{table_id}` t
    cross join unnest(t.prediction_result) t_pred
    where (t.prediction_date>='{start_date}' and  t.prediction_date<='{end_date}')
    and t.model_id='{model_id}'
    order by  t.prediction_date,t_pred.output_date
    """
    print(sqlOutput)
    dfOutput=load_data_bq(sqlOutput)
    # dfOutput=dfOutput.drop_duplicates(subset=[date_col,'asset_name','prediction_name'],keep='last',)
    # dfOutput=dfOutput.drop_duplicates(subset=[date_col],keep='last',)
    dfOutput[date_col]=pd.to_datetime(dfOutput[date_col],format='%Y-%m-%d')
    dfOutput.set_index(date_col,inplace=True)

    output_sequence_length=len(dfOutput)
    print(f"output_sequence_length={output_sequence_length}")
    

    print(dfOutput.info())
    print(dfOutput[['prediction_date','asset_name','prediction_name' ,prediction_name]])
    print("================================================================================================")

    
    #get actual data since the fist day of input and the last day of output(if covered)
    startFinData=dfOutput.index.min().strftime('%Y-%m-%d')
    endFindData=dfOutput.index.max().strftime('%Y-%m-%d')
    print(f"2.Get Real Data  to compare to prediction from {startFinData} to {endFindData}")

    sqlData=f"""
    select Date as {date_col},{prediction_name}, ImportDateTime, from `{table_data_id}` 
    where (Date>='{startFinData}' and Date<='{endFindData}') and Symbol='{asset_name}'
    order by ImportDateTime,Date
    """
    print(sqlData)

    dfRealData=load_data_bq(sqlData)
    dfRealData=dfRealData.drop_duplicates(subset=[date_col],keep='last',)
    dfRealData[date_col]=pd.to_datetime(dfRealData[date_col],format='%Y-%m-%d')
    dfRealData.set_index(date_col,inplace=True)
    
    print(dfRealData.info())
    print(dfRealData[[prediction_name]])
    print("================================================================================================")

    return {'actual_price':dfRealData,'output':dfOutput }


print(f"================Get data from {startX}====to==={endX}================")
request={'start_date':startX,'end_date':endX,'prediction_name':prediction,'asset_name':asset_name,'model_id':model_id}
data=get_forecasting_result_data(request)
print(f"=======================================================================")

================Get data from 2023-08-28====to===2023-09-01================
1.How far in advance does model want to  make prediction

    select t.prediction_date, t.pred_timestamp,t.asset_name,t.prediction_name,
    t_pred.output_date as date,t_pred.output_value as EMA1
    from  `pongthorn.FinAssetForecast.fin_movement_forecast` t
    cross join unnest(t.prediction_result) t_pred
    where (t.prediction_date>='2023-08-28' and  t.prediction_date<='2023-09-01')
    and t.model_id='qqq-ema1-30t5-ds0115t0523'
    order by  t.prediction_date,t_pred.output_date
    
output_sequence_length=25
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 25 entries, 2023-08-29 to 2023-09-11
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype              
---  ------           --------------  -----              
 0   prediction_date  25 non-null     dbdate             
 1   pred_timestamp   25 non-null     datetime64[us, UTC]
 2   asset_name       25 non-null     object       

# Create Predictive and Actual Value dataframe

In [134]:
print("List all trading day in the week")
myTradingDataList=data['output']['prediction_date'].unique()
print(myTradingDataList)

List all trading day in the week
<DateArray>
[datetime.date(2023, 8, 28), datetime.date(2023, 8, 29),
 datetime.date(2023, 8, 30), datetime.date(2023, 8, 31),
  datetime.date(2023, 9, 1)]
Length: 5, dtype: dbdate


In [135]:
dfAllForecastResult=pd.DataFrame(columns=['date','pred_value','actual_value','prediction_date'])
dfAllForecastResult

,date,pred_value,actual_value,prediction_date


In [136]:
print(f"========================dfX :Actual Price========================")
dfX=data['actual_price'][[prediction]]
dfX.columns=[f'actual_value']
print(dfX.info())
dfX

========================dfX :Actual Price========================
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5 entries, 2023-08-29 to 2023-09-05
Data columns (total 1 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   actual_value  5 non-null      float64
dtypes: float64(1)
memory usage: 80.0 bytes
None


,actual_value
date,
2023-08-29,366.8853
2023-08-30,368.6989
2023-08-31,370.3882
2023-09-01,371.7010
2023-09-05,372.8464


In [137]:
# actually , we can jon without spilting data by prediction_dtate
for date in  myTradingDataList: # trading day on giver week
    print(f"=========================dfPred:Predicted Price at {date}=========================")
    dfPred=data['output'].query("prediction_date==@date")[[prediction]]
    dfPred.columns=[f'pred_value']
    print(dfPred)
    print(dfPred.info())

    print("=====================dfCompare:Join Actual price to Predicted Price=================")
    dfCompare=pd.merge(left=dfPred,right=dfX,how='inner',right_index=True,left_index=True)
    dfCompare.reset_index(inplace=True)   
    dfCompare['prediction_date']=date.strftime('%Y-%m-%d')      
    print(dfCompare) 
    print(dfCompare.info())

    if len(dfCompare)>0 : # it will be join if there is at least one record to show actual vs pred
        dfAllForecastResult= pd.concat([dfAllForecastResult,dfCompare],ignore_index=True)
        print(f"=========================Appended Data Joined=========================")
    else:
        print("No Appendind Data due to no at least one record to show actual vs pred")  
    


=========================dfPred:Predicted Price at 2023-08-28=========================
            pred_value
date                  
2023-08-29  367.377289
2023-08-30  368.779694
2023-08-31  365.557068
2023-09-01  364.746979
2023-09-05  364.759613
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5 entries, 2023-08-29 to 2023-09-05
Data columns (total 1 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   pred_value  5 non-null      float64
dtypes: float64(1)
memory usage: 80.0 bytes
None
=====================dfCompare:Join Actual price to Predicted Price=================
        date  pred_value  actual_value prediction_date
0 2023-08-29  367.377289      366.8853      2023-08-28
1 2023-08-30  368.779694      368.6989      2023-08-28
2 2023-08-31  365.557068      370.3882      2023-08-28
3 2023-09-01  364.746979      371.7010      2023-08-28
4 2023-09-05  364.759613      372.8464      2023-08-28
<class 'pandas.core.frame.DataFrame'>
RangeI

In [138]:
print("========================dfAllForecastResult: All Predicton Result========================")
print(dfAllForecastResult.info())
print(dfAllForecastResult)

========================dfAllForecastResult: All Predicton Result========================
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   date             15 non-null     datetime64[ns]
 1   pred_value       15 non-null     float64       
 2   actual_value     15 non-null     float64       
 3   prediction_date  15 non-null     object        
dtypes: datetime64[ns](1), float64(2), object(1)
memory usage: 608.0+ bytes
None
         date  pred_value  actual_value prediction_date
0  2023-08-29  367.377289      366.8853      2023-08-28
1  2023-08-30  368.779694      368.6989      2023-08-28
2  2023-08-31  365.557068      370.3882      2023-08-28
3  2023-09-01  364.746979      371.7010      2023-08-28
4  2023-09-05  364.759613      372.8464      2023-08-28
5  2023-08-30  369.908997      368.6989      2023-08-29
6  2023-08-31  37

# Calculate MAE Metric

## Get sum distance between pred and actul value from prev rows

In [139]:
sqlMetric=f"""
with pred_actual_by_model as  
(
SELECT  detail.actual_value,detail.pred_value
from `{table_perf_id}`  t
 cross join unnest(t.pred_actual_data) as detail
 where t.model_id='{model_id}' and t.collection_timestamp<'{log_timestamp}'
)
select COALESCE( sum(abs(x.actual_value-x.pred_value)),0) as pred_diff_actual,count(*) as no_row  from pred_actual_by_model  x


"""

if genTableSchema==False:
    print(sqlMetric)

    dfMetric=load_data_bq(sqlMetric)
    prevSum=dfMetric.iloc[0,0]
    prevCount=dfMetric.iloc[0,1]

else:  # it is used if there are something changed in table schema
# for generating table schema
    prevSum=0
    prevCount=0

print(f"Prev Sum={prevSum} and Count={prevCount}")

Prev Sum=0 and Count=0


## Cal sum distance between pred and actul value from last rows

In [140]:
dfAllForecastResult['pred_diff_actual']=dfAllForecastResult.apply(lambda x : abs(x['pred_value']-x['actual_value']),axis=1)
dfAllForecastResult

,date,pred_value,actual_value,prediction_date,pred_diff_actual
0,2023-08-29,367.377289,366.8853,2023-08-28,0.491989
1,2023-08-30,368.779694,368.6989,2023-08-28,0.080794
2,2023-08-31,365.557068,370.3882,2023-08-28,4.831132
3,2023-09-01,364.746979,371.7010,2023-08-28,6.954021
4,2023-09-05,364.759613,372.8464,2023-08-28,8.086787
5,2023-08-30,369.908997,368.6989,2023-08-29,1.210097
6,2023-08-31,371.342102,370.3882,2023-08-29,0.953902
7,2023-09-01,368.157196,371.7010,2023-08-29,3.543804
8,2023-09-05,367.743347,372.8464,2023-08-29,5.103053
9,2023-08-31,373.007904,370.3882,2023-08-30,2.619704


In [141]:
recentSum=dfAllForecastResult['pred_diff_actual'].sum()
recentCount=len(dfAllForecastResult)


dfAllForecastResult=dfAllForecastResult.drop(columns=['pred_diff_actual'])
print(f"Recent Sum={recentSum} and Count={recentCount}")

Recent Sum=48.47301259750003 and Count=15


## Calculate MEA as formula

In [142]:
#https://en.wikipedia.org/wiki/Mean_absolute_error
metric_value= round((prevSum+recentSum)/(prevCount+recentCount),2)
print(f"{metric_name} = {metric_value}")

mae = 3.23


# Create Collection Performance Info Dataframe and Store 


In [143]:
df=pd.DataFrame(data=[ [log_date,model_id,metric_name,metric_value,log_timestamp] ],
                columns=["collection_date","model_id","metric_name","metric_value","collection_timestamp"])
print(df.info())
print(df)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 5 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   collection_date       1 non-null      datetime64[ns]
 1   model_id              1 non-null      object        
 2   metric_name           1 non-null      object        
 3   metric_value          1 non-null      float64       
 4   collection_timestamp  1 non-null      datetime64[ns]
dtypes: datetime64[ns](2), float64(1), object(2)
memory usage: 168.0+ bytes
None
  collection_date                   model_id metric_name  metric_value  \
0      2023-09-02  qqq-ema1-30t5-ds0115t0523         mae          3.23   

  collection_timestamp  
0           2023-09-02  


In [144]:
dictCollectPerf[model_id]=(df,dfAllForecastResult)

In [145]:
# uncomment
#return f"gather data of {model_id}"


# End Loop

In [146]:
# Iterate over model list
# uncomment
# for modelID in modelList:
# # indent
#   print(process_data(modelID))
#   print("#########################################################")

# Create Json Data 

In [147]:
jsonDataList=[]
for model_id,dataTuple in  dictCollectPerf.items():
    print(model_id)
    
    masterDF=dataTuple[0]
    masterDF["collection_date"]=masterDF["collection_date"].dt.strftime('%Y-%m-%d')
    masterDF["collection_timestamp"]=masterDF["collection_timestamp"].dt.strftime('%Y-%m-%d %H:%M:%S')
    master_perf = json.loads(masterDF.to_json(orient = 'records'))[0] # 1 main dataframe has 1 records
    
    detailDF=dataTuple[1]  
    # print(detailDF.info())
    
    #detailDF["prediction_date"]=detailDF["prediction_date"].dt.strftime('%Y-%m-%d')
    detailDF["date"]=detailDF["date"].dt.strftime('%Y-%m-%d')
    
    detail_perf= json.loads(detailDF.to_json(orient = 'records'))
    master_perf["pred_actual_data"]=detail_perf
    
    jsonDataList.append(master_perf)
    
with open("fin_forecast_performance.json", "w") as outfile:
    json.dump( jsonDataList, outfile)

qqq-ema1-30t5-ds0115t0523


# Ingest Data to BigQuery

## Try to ingest data to get correct schema and copy the schema to create table including partion/cluster manually

In [150]:
try:
    table=client.get_table(table_perf_id)
    print("Table {} already exists.".format(table_perf_id))
    # print(table.schema)
except Exception as ex :
    print(str(ex))

Table pongthorn.FinAssetForecast.model2_demo_forecast_performance already exists.


In [151]:

job_config = bigquery.LoadJobConfig()

job_config.source_format = bigquery.SourceFormat.NEWLINE_DELIMITED_JSON

# Try to ingest data to get correct schema and copy the schema to create table including partiion/cluster manually
job_config.write_disposition = bigquery.WriteDisposition.WRITE_APPEND 


job = client.load_table_from_json(jsonDataList,table_perf_id, job_config = job_config)
if job.errors is not None:
    print(job.error_result)
    print(job.errors)
    # uncomment
    # return "Error to load data to BigQuery"
else:
    print(f"Import to bigquery successfully  {len(jsonDataList)} records")
    
#job_config.schema
# truncate table`pongthorn.FinAssetForecast.model_forecast_performance` 

Import to bigquery successfully  1 records


In [27]:
# uncomment
#return 'completely'

In [4]:
# uncomment
# Main 
# print("Collect prediction result to monitor performance model")
start_backfill='2023-06-03 10:00' # comment
end_backfill='2023-08-26 10:00'
period_index=pd.date_range(start=start_backfill,end=end_backfill, freq="W-SAT")
listLogDate=[ d.strftime('%Y-%m-%d %H:%M')   for  d in  period_index   ]
for d in listLogDate:
    print(d)
# multiple items

# listLogDate=[
#      '2023-08-05 00:00','2023-08-12 00:00','2023-08-19 00:00','2023-08-26 00:00','2023-09-02 00:00'
# ]
# for  d in listLogDate:
#   print(f"*******************************Collect prediction result as of {d}*****************************************")
#   print(collect_prediction_result(d))
#   print("************************************************************************************************")

# sigle item
# collectionDate='2023-08-26 00:00' # comment    
# print(collect_prediction_result(collectionDate))


2023-06-03 10:00
2023-06-10 10:00
2023-06-17 10:00
2023-06-24 10:00
2023-07-01 10:00
2023-07-08 10:00
2023-07-15 10:00
2023-07-22 10:00
2023-07-29 10:00
2023-08-05 10:00
2023-08-12 10:00
2023-08-19 10:00
2023-08-26 10:00
